In [1]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to select a file from your computer


Saving activity_log.txt to activity_log.txt


In [2]:
!ls


activity_log.txt  sample_data


In [3]:
import re
from collections import defaultdict

# -------------------------------
# Student Class
# -------------------------------
class Student:
    def __init__(self, student_id, name):
        self.student_id = student_id
        self.name = name
        self.activities = []

    def add_activity(self, activity, date, time):
        self.activities.append((activity, date, time))

    def display_summary(self):
        logins = sum(1 for a in self.activities if a[0] == "LOGIN")
        submissions = sum(1 for a in self.activities if a[0] == "SUBMIT_ASSIGNMENT")
        return logins, submissions


# -------------------------------
# Generator to read log file
# -------------------------------
def read_log_file(filename):
    with open(filename, "r") as file:
        for line in file:
            try:
                parts = [p.strip() for p in line.split("|")]
                if len(parts) != 5:
                    raise ValueError("Invalid format")

                student_id, name, activity, date, time = parts

                # Regex validation
                if not re.match(r"^S\d+$", student_id):
                    raise ValueError("Invalid Student ID")

                if activity not in ["LOGIN", "LOGOUT", "SUBMIT_ASSIGNMENT"]:
                    raise ValueError("Invalid Activity")

                if not re.match(r"\d{4}-\d{2}-\d{2}", date):
                    raise ValueError("Invalid Date")

                if not re.match(r"\d{2}:\d{2}", time):
                    raise ValueError("Invalid Time")

                yield student_id, name, activity, date, time

            except Exception as e:
                print(f"Invalid log entry skipped: {line.strip()}")


# -------------------------------
# Main Processing
# -------------------------------
students = {}
daily_activity = defaultdict(int)
login_tracker = defaultdict(int)

# Input file (example: activity_log.txt)
input_file = "activity_log.txt"
output_file = "activity_report.txt"

for sid, name, activity, date, time in read_log_file(input_file):
    if sid not in students:
        students[sid] = Student(sid, name)

    students[sid].add_activity(activity, date, time)
    daily_activity[date] += 1

    # Abnormal behavior detection
    if activity == "LOGIN":
        login_tracker[sid] += 1
    elif activity == "LOGOUT":
        login_tracker[sid] = max(0, login_tracker[sid] - 1)


# -------------------------------
# Report Generation
# -------------------------------
with open(output_file, "w") as out:
    print("STUDENT ACTIVITY REPORT")
    print("-" * 50)
    out.write("STUDENT ACTIVITY REPORT\n")
    out.write("-" * 50 + "\n")

    for student in students.values():
        logins, submissions = student.display_summary()

        report = (
            f"Student ID   : {student.student_id}\n"
            f"Name         : {student.name}\n"
            f"Total Logins : {logins}\n"
            f"Submissions  : {submissions}\n"
        )

        print(report)
        out.write(report + "\n")

    print("DAILY ACTIVITY STATISTICS")
    print("-" * 50)
    out.write("DAILY ACTIVITY STATISTICS\n")
    out.write("-" * 50 + "\n")

    for date, count in daily_activity.items():
        line = f"{date} : {count} activities\n"
        print(line.strip())
        out.write(line)

    print("\nABNORMAL BEHAVIOR DETECTED")
    print("-" * 50)
    out.write("\nABNORMAL BEHAVIOR DETECTED\n")
    out.write("-" * 50 + "\n")

    for sid, count in login_tracker.items():
        if count > 0:
            warning = f"{sid} has multiple logins without logout\n"
            print(warning.strip())
            out.write(warning)

print("\nReport saved to", output_file)


Invalid log entry skipped: INVALID | ENTRY | LOGIN | 2025-03-11 | 09:00
Invalid log entry skipped: S105 | Sam | INVALID_ACTIVITY | 2025-03-12 | 08:00
STUDENT ACTIVITY REPORT
--------------------------------------------------
Student ID   : S101
Name         : Asha
Total Logins : 2
Submissions  : 1

Student ID   : S102
Name         : Ravi
Total Logins : 1
Submissions  : 1

Student ID   : S103
Name         : Meena
Total Logins : 1
Submissions  : 1

Student ID   : S104
Name         : Kiran
Total Logins : 2
Submissions  : 0

DAILY ACTIVITY STATISTICS
--------------------------------------------------
2025-03-10 : 3 activities
2025-03-11 : 10 activities

ABNORMAL BEHAVIOR DETECTED
--------------------------------------------------
S103 has multiple logins without logout
S104 has multiple logins without logout

Report saved to activity_report.txt
